In [1]:
import pandas as pd
import numpy as np

from nltk.stem import PorterStemmer
stop_words = ['the', 'and', 'to', 'of', 'is', 'in', 'that', 'it', 'with', 'for', 'on', 'was', 'as', 'at', 'by', 'an', 'be', 'this', 'which', 'or', 'from', 'not', 'but',
              'are', 'they', 'if', 'we', 'all', 'can', 'more', 'will', 'has', 'their', 'its', 'who', 'than', 'then', 'had', 'her', 'him', 'she', 'he', 'my', 'your', 'our', 'us']

df = pd.read_csv('scp6999.csv')
print("Successfully read data from CSV.")
print("Now cleaning the data...")
# cleaning up data, all text including and after this is not part of the article.
df['text'] = df['text'].str.split('« SCP').str[0]
df['text'] = df['text'].replace('[^a-zA-Z\s]', '', regex=True)
df['text'] = df['text'].replace('  ', ' ', regex=True)
df['text'] = df['text'].str.lower().apply(lambda x: ' '.join(
        [word for word in x.split() if word not in stop_words]))  # remove stop words and put to lowercase
df['Euclid'] = df['Euclid'].fillna('none/other')
stemmer = PorterStemmer()
df['text_stemmed'] = df['text'].apply(
        lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))  # convert words to their stemmed form

Successfully read data from CSV.
Now cleaning the data...


In [2]:
from collections import Counter


def bag_of_words(texts):
    """
    Computes a bag of words representation for a list of texts
    """
    # Preprocess the texts
    preprocessed_texts = texts

    # Create a dictionary of word frequencies
    word_freqs = Counter()
    for text in preprocessed_texts:
        words = text.split()
        word_freqs.update(words)

    # Create a list of unique words
    unique_words = list(word_freqs.keys())

    # Create a dictionary mapping words to indices
    word_to_index = {word: i for i, word in enumerate(unique_words)}

    # Create a matrix of word frequencies
    num_texts = len(preprocessed_texts)
    num_words = len(unique_words)
    word_freq_matrix = [[0 for j in range(num_words)] for i in range(num_texts)]
    for i, text in enumerate(preprocessed_texts):
        words = text.split()
        for word in words:
            word_index = word_to_index[word]
            word_freq_matrix[i][word_index] += 1

    return word_freq_matrix, unique_words

In [3]:

print("Data has been loaded and cleaned!")
print("Building features...")
X, unique_words = bag_of_words(df['text'])

print("Word frequency features have been built.")

Data has been loaded and cleaned!
Building features...
Word frequency features have been built.


In [4]:
from sklearn.model_selection import train_test_split


    # df.to_csv("../data/processed/freq.csv", index=False)
    # print("Data has been saved to data/processed/freq.csv")
    # df = pd.read_csv("../data/processed/freq.csv")
y = df['Euclid']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
param_grid = {'n_estimators': [500, 1000, 2000, 4000, 120000]}
grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_) 
confusion_matrix(y_test, grid.best_estimator_.predict(X_test))


/Users/daniel/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
rf.score(X_test, y_test)

NameError: name 'rf' is not defined